<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Get-Data" data-toc-modified-id="Get-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get Data</a></span></li><li><span><a href="#Profile-the-Dataset" data-toc-modified-id="Profile-the-Dataset-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Profile the Dataset</a></span></li><li><span><a href="#Clean-the-Data" data-toc-modified-id="Clean-the-Data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Clean the Data</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#-Subset-on-City-of-San-Francisco-Only" data-toc-modified-id="-Subset-on-City-of-San-Francisco-Only-4.0.1"><span class="toc-item-num">4.0.1&nbsp;&nbsp;</span> Subset on City of San Francisco Only</a></span></li><li><span><a href="#Drop-Unneeded-Columns" data-toc-modified-id="Drop-Unneeded-Columns-4.0.2"><span class="toc-item-num">4.0.2&nbsp;&nbsp;</span>Drop Unneeded Columns</a></span></li><li><span><a href="#Modify-Date-Fields" data-toc-modified-id="Modify-Date-Fields-4.0.3"><span class="toc-item-num">4.0.3&nbsp;&nbsp;</span>Modify Date Fields</a></span></li><li><span><a href="#Save" data-toc-modified-id="Save-4.0.4"><span class="toc-item-num">4.0.4&nbsp;&nbsp;</span>Save</a></span></li></ul></li></ul></li></ul></div>

<a id="section__top"></a>

<h2>Imports

In [2]:
import pandas as pd
import numpy as np
import pandas_profiling

import os
import sys
import re

module_path = os.path.abspath(os.path.join('./lib/'))

if module_path not in sys.path:   
    sys.path.append(module_path)

from utilities import *
from sodapy_dataset_reader import *


import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline

/Users/anne/dsi2/Survival Analysis/code/lib


The sodapy_dataset_reader import contains a classes that use the
__[sodapy library ](https://pypi.org/project/sodapy/)__
to interface with the __[Socrata API ](https://dev.socrata.com)__
<br>
Many open data platforms are implemented using the Socrata (or SODA) API, including
__[SFData.gov](https://data.sfgov.org/)__, which is used for this project.

<h2>Get Data

In [3]:
#data source
source_domain = 'data.sfgov.org'
#dataset registered_businesses_dataset
dataset_nickname = 'reg_bus'
dataset_id = 'g8m3-pdis'

#Instantiate Reader
spr = SodapyDatasetReader(source_domain, dataset_id)

#Get Meta Data
md = spr.get_metadata()

#get and print rowcount
rowcount = int(spr.get_row_count()[0]['count'])
print(f'Dataset contains {rowcount} rows')

#set limit to rowcount
df = spr.get_df(limit=rowcount)
df.shape

Dataset contains 238523 rows


(238523, 36)

Uncomment the following cell to display markdown description of the dataset.  It is quite long, so not rendered, but here is a sample of the output: 

```

{'id': 'g8m3-pdis',
 'name': 'Registered Business Locations - San Francisco',
 'attribution': 'City and County of San Francisco',
 'attributionLink': 'http://www.sfgov.org',
 'averageRating': 0,
 'category': 'Economy and Community',
 'createdAt': 1389898067,
 'description': 'This dataset includes the locations of businesses that pay taxes to the City and County of San Francisco. Each registered business may have multiple locations and each location is a single row. The Treasurer & Tax Collector’s Office collects this data through business registration applications, account update/closure forms, and taxpayer filings. The data is collected to help enforce the Business and Tax Regulations Code including, but not limited to: Article 6, Article 12, Article 12-A, and Article 12-A-1. http://sftreasurer.org/registration',
 'displayType': 'table',
 'downloadCount': 30187,
 'hideFromCatalog': False,
 
 
 ```

In [10]:
#md

<h2>Profile the Dataset

The Pandas profiler is a utility function that expands on the initial data exploration 
steps one does when one first loads a new dataset. Code and documentation can be found here:
__[Pandas Profiling](https://github.com/pandas-profiling/pandas-profiling)__  <br><br> This process can be resource intensive, so after having run it once and saved the output, I chose to comment it out, but leave the code available for you to uncomment if you chose.

In [13]:
#profile = pandas_profiling.ProfileReport(df, check_correlation=False)

In [14]:
#display(profile)

In [15]:
#output to file...
#fn = f'../tmp/profile-{dataset_nickname}-{dataset_id}.html'
#profile.to_file(fn)

<h2>Clean the Data

[back to top](#section__top)

In [16]:
df.head()

,:@computed_region_26cr_cadq,:@computed_region_6qbp_sg9q,:@computed_region_ajp5_b2md,:@computed_region_bh8s_q3mv,:@computed_region_fyvs_ahh9,:@computed_region_jx4q_fizf,:@computed_region_p5aj_wyqh,:@computed_region_qgnn_b9vv,:@computed_region_rxqg_mtj9,:@computed_region_yftq_j783,...,mailing_address_1,naic_code,naic_code_description,neighborhoods_analysis_boundaries,ownership_name,parking_tax,state,supervisor_district,transient_occupancy_tax,ttxid
0,NaN,NaN,NaN,26991,NaN,NaN,NaN,NaN,NaN,NaN,...,343 State St,3100-3399,Manufacturing,NaN,Eastman Kodak Co,False,NY,NaN,False,0000024-02-999
1,6,15,13,57,17,6,9,4,1,13,...,2799 California St,5300-5399,Real Estate and Rental and Leasing Services,Marina,Ferrando Louise,False,CA,2,False,0000028-02-001
2,8,45,41,29491,40,5,8,10,4,1,...,8 Mendosa Ave,5300-5399,Real Estate and Rental and Leasing Services,West of Twin Peaks,Ideal Novak Corp,False,CA,7,False,0000052-01-001
3,6,17,13,57,17,6,9,4,1,13,...,4444 Geary Blvd Ste 100,NaN,NaN,NaN,Tournahu George L,False,CA,NaN,False,0000071-01-001
4,6,17,13,57,17,6,9,4,1,13,...,4444 Geary Blvd Ste 100,NaN,NaN,Marina,Tournahu George L,False,CA,2,False,0000071-02-001


<h4> Subset on City of San Francisco Only

Look city column. Notice that there are some business registrations for locations outside of San Francisco, and there are some entires where San Francisco was misspelled.

In [17]:
df.city.value_counts().head()

San Francisco    178819
San+francisco     16921
Oakland            3008
Daly+city          1855
San+jose           1337
Name: city, dtype: int64

In [18]:
bad_cities = ['San francisco', 'San Fransicsco', 'San Rancisco', 'San fracnscio', 'San Frascisco']

In [19]:
df['city'] = df['city'].str.replace('+',' ')
df['city'] = df['city'].map(lambda x: 'San Francisco' if x in bad_cities else x)

In [20]:
df['city'].value_counts().head()

San Francisco    195744
Oakland            3008
Daly city          1855
San jose           1337
Hayward             929
Name: city, dtype: int64

In [21]:
sfmask = df['city'] == 'San Francisco'

In [22]:
dfsf = df[sfmask]

In [23]:
dfsf.city.value_counts()

San Francisco    195744
Name: city, dtype: int64

In [24]:
dfsf.shape

(195744, 36)

<h4>Drop Unneeded Columns

In [25]:
dfsf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195744 entries, 1 to 238522
Data columns (total 36 columns):
:@computed_region_26cr_cadq          141709 non-null object
:@computed_region_6qbp_sg9q          141587 non-null object
:@computed_region_ajp5_b2md          141705 non-null object
:@computed_region_bh8s_q3mv          141926 non-null object
:@computed_region_fyvs_ahh9          141705 non-null object
:@computed_region_jx4q_fizf          141176 non-null object
:@computed_region_p5aj_wyqh          141176 non-null object
:@computed_region_qgnn_b9vv          141198 non-null object
:@computed_region_rxqg_mtj9          141709 non-null object
:@computed_region_yftq_j783          141212 non-null object
business_corridor                    324 non-null object
business_zip                         195624 non-null object
certificate_number                   195744 non-null object
city                                 195744 non-null object
dba_end_date                         66511 non-null 

In [26]:
dfsf.shape

(195744, 36)

drop computed columns per this link
https://support.socrata.com/hc/en-us/articles/360007155973-Handling-Computed-Columns-with-FME
Datasets that have location data that intersects a Spatial Lens Boundary will have computed columns. These columns are created and curated by the Socrata platform, not the user. They are used to perform the geographic join between the geocoded row and the underlying spatial lens polygon. The column names begin with the prefix :@computed_region_.

In [27]:
dfsf = dfsf.drop([':@computed_region_26cr_cadq', ':@computed_region_6qbp_sg9q',
       ':@computed_region_ajp5_b2md', ':@computed_region_bh8s_q3mv',
       ':@computed_region_fyvs_ahh9', ':@computed_region_jx4q_fizf',
       ':@computed_region_p5aj_wyqh', ':@computed_region_qgnn_b9vv',
       ':@computed_region_rxqg_mtj9', ':@computed_region_yftq_j783'], axis = 1)

In [28]:
dfsf.shape

(195744, 26)

In [29]:
dfsf.isnull().sum()

business_corridor                    195420
business_zip                            120
certificate_number                        0
city                                      0
dba_end_date                         129233
dba_name                                  0
dba_start_date                            0
full_business_address                     2
lic                                  181327
lic_code_description                 181875
location                              51861
location_end_date                    108324
location_start_date                       0
mail_city                             64306
mail_state                            66548
mail_zipcode                          64319
mailing_address_1                     66511
naic_code                             73597
naic_code_description                 73597
neighborhoods_analysis_boundaries     45904
ownership_name                            0
parking_tax                               0
state                           

In [30]:
#drop business corridor - mostly missing
dfsf = dfsf.drop(['business_corridor'],axis=1)

In [31]:
dfsf.shape

(195744, 25)

In [32]:
dfsf.columns

Index(['business_zip', 'certificate_number', 'city', 'dba_end_date',
       'dba_name', 'dba_start_date', 'full_business_address', 'lic',
       'lic_code_description', 'location', 'location_end_date',
       'location_start_date', 'mail_city', 'mail_state', 'mail_zipcode',
       'mailing_address_1', 'naic_code', 'naic_code_description',
       'neighborhoods_analysis_boundaries', 'ownership_name', 'parking_tax',
       'state', 'supervisor_district', 'transient_occupancy_tax', 'ttxid'],
      dtype='object')

<h4>Modify Date Fields

The data contains two sets of start and end dates. "DBA" is the "doing business as" date. Many more DBA end dates are missing than location end dates. Since it is the location of the business itself we are interested here, we use loc_start and loc_end for the analysis, but keep dba in for reference. 

In [33]:
#create new columns for dba 'doing business as' and location start and stop
#to contain just the dates
dfsf['dba_start'] = pd.to_datetime(dfsf['dba_start_date']) 
dfsf['dba_end'] = pd.to_datetime(dfsf['dba_end_date'])
dfsf['loc_start'] = pd.to_datetime(dfsf['location_start_date'])
dfsf['loc_end'] = pd.to_datetime(dfsf['location_end_date'])

In [34]:
dfsf.isnull().sum()

business_zip                            120
certificate_number                        0
city                                      0
dba_end_date                         129233
dba_name                                  0
dba_start_date                            0
full_business_address                     2
lic                                  181327
lic_code_description                 181875
location                              51861
location_end_date                    108324
location_start_date                       0
mail_city                             64306
mail_state                            66548
mail_zipcode                          64319
mailing_address_1                     66511
naic_code                             73597
naic_code_description                 73597
neighborhoods_analysis_boundaries     45904
ownership_name                            0
parking_tax                               0
state                                    14
supervisor_district             

In [35]:
#then drop the original date/time columns
dfsf = dfsf.drop(['dba_start_date', 'dba_end_date', 'location_start_date', 'location_end_date'] , axis=1)

<h4>Save

Save the data in the tmp directory. It will be combined with other data to create the final dataset for analysis.

In [36]:
ts = get_timestamp()
dfsf.to_csv(f'../tmp/reg_bus_sfonly_raw{ts}.csv', index=False)
